In [41]:
import re
import os
import pymupdf
import pandas as pd
from langchain.document_loaders import PyPDFLoader

Creating pandas dataframe with necessary columns

In [42]:
columns_01 = ['date','nea', 'nea_subsidiary', 'ipp', 'import', 'total_energy_available', 
          'energy_export', 'inps_demand', 'energy_interruption', 
          'generation_deficit', 'energy_requirement', 'net_exchange_with_india']
daily_energy_df=pd.DataFrame(columns=columns_01)

columns_02 = ['date','pk_time','generation','import', 'recorded_pk_availablity', 'export', 'demd_met@pk_time',
               'interruption', 'deficit', 'pk_demd_met', 'net_exchange_with_india']
exchange_at_peak_time_df=pd.DataFrame(columns=columns_02)

columns_03 = ['date','sum_of_import_energy','sum_of_export_energy','net_exchange_with_india']
energy_exchange_df=pd.DataFrame(columns=columns_03)

Extracting date from pdf using PyPDFLoader and Python Regex

In [43]:
def get_date(file_path):
    loader = PyPDFLoader(file_path)
    page = loader.load()[0]
    page_content = page.page_content
    pattern = r'\(\s*(\d{4}/\d{1,2}/\d{1,2})\s*\)'
    match_date = re.search(pattern,page_content)

    if match_date:
        date = match_date.group(1)
        return date
    else:
        date=""
        return date

Function to extract all the information from the first table and appending data to the *daily_energy_df*

In [44]:
def get_daily_energy(file_path):
    doc=pymupdf.open(file_path)
    page_one=doc[0]
    find=page_one.find_tables()
    table=find.tables[0]
    daily_energy=table.extract()[3]
    daily_energy = [x for x in daily_energy if x is not None]
    if len(daily_energy) < 11:
        daily_energy.extend([0] * (11 - len(daily_energy)))
    daily_energy.insert(0,get_date(file_path))

    daily_energy_df.loc[len(daily_energy_df)] = daily_energy
    return daily_energy_df

Function to extract all the information from the second table and append data to the *exchange_at_peak_time_df*

In [45]:
def get_exchange_at_peak_time(file_path):
    doc=pymupdf.open(file_path)
    page_one=doc[0]
    find=page_one.find_tables()
    table=find.tables[0]
    exchange_at_peak_time=table.extract()[7]
    exchange_at_peak_time = [x for x in exchange_at_peak_time if x is not None]
    exchange_at_peak_time.insert(0,get_date(file_path))

    exchange_at_peak_time_df.loc[len(exchange_at_peak_time_df)] = exchange_at_peak_time
    return exchange_at_peak_time_df

Function to extract the information from the third table and append data to the *energy_exchange_df*

In [46]:
def get_energy_exchange(file_path):
    doc=pymupdf.open(file_path)
    page_one=doc[0]
    find=page_one.find_tables()
    table=find.tables[0]
    try:
        energy_exchange=table.extract()[11]
        energy_exchange = [x for x in energy_exchange if x is not None][1:]
    except Exception:
        energy_exchange=[0,0,0]

    energy_exchange.insert(0,get_date(file_path))
    energy_exchange_df.loc[len(energy_exchange_df)] = energy_exchange
    return energy_exchange_df

Function to check if the pdf is valid or not

In [47]:
def is_valid_pdf(file_path):
    try:
        with open(file_path, 'rb') as f:
            header = f.read(5)
            return header == b'%PDF-'
    except Exception:
        return False

Finally calling all the functions to:
1. Process each PDF file in the 'pdfs' folder
2. Extract daily energy values, exchange at peak time, and energy exchange data
3. Skip invalid PDFs and handle errors
4. Save the extracted data to CSV files

In [48]:
pdf_folder = "pdfs"

for pdf_file in os.listdir(pdf_folder):
    if not pdf_file.endswith('.pdf'):
        continue

    file_path = os.path.join(pdf_folder, pdf_file)

    # Skip invalid PDFs
    if not is_valid_pdf(file_path):
        print(f"Skipping invalid PDF: {pdf_file}")
        continue
    try:
        get_daily_energy(file_path)
        get_exchange_at_peak_time(file_path)
        get_energy_exchange(file_path)
        print(f"Processed {pdf_file}")
    except Exception as e:
        print(f"Error processing {pdf_file}: {e}")


Processed 13101186.pdf
Processed 15761410.pdf
Processed 19571562.pdf
Processed 23838484.pdf
Processed 29789573.pdf
Processed 30850636.pdf
Processed 32858115.pdf
Processed 34460951.pdf
Processed 38885017.pdf
Processed 39772590.pdf
Processed 42728780.pdf
Processed 43964675.pdf
Processed 51762050.pdf
Processed 51855480.pdf
Processed 58932630.pdf
Processed 60795644.pdf
Processed 63226566.pdf
Processed 65538523.pdf
Processed 65768332.pdf
Processed 67827024.pdf
Processed 68227586.pdf
Processed 69182469.pdf
Processed 71034072.pdf
Processed 73095374.pdf
Processed 80360500.pdf
Processed 81843544.pdf
Processed 82817584.pdf
Processed 82962885.pdf
Processed 87807308.pdf
Processed 88117435.pdf
Processed 95236178.pdf
Processed 95538096.pdf
Processed 96010606.pdf
Processed 98534245.pdf
Processed 99899849.pdf
Processed NDOR%202079_06_24.pdf
Processed NDOR%202079_06_25.pdf
Skipping invalid PDF: NDOR%202079_06_26.pdf
Processed NDOR%202079_06_27.pdf
Processed NDOR%202079_06_28.pdf
Processed NDOR%202079_0

In [57]:
daily_energy_df.to_csv("./csv_files/daily_energy_values.csv", index=False)
exchange_at_peak_time_df.to_csv("./csv_files/demand_cb_exchange@peak_time.csv", index=False)
energy_exchange_df.to_csv("./csv_files/energy_ex_cb_line.csv", index=False)
